<a href="https://colab.research.google.com/github/guilhermelaviola/IntelligentCommunication/blob/main/Class09.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **User Management**
User management in chatbots is crucial for providing an interactive and personalized experience. It involves creating a data model that includes user information like ID, name, preferences, and interaction history. This data must be stored securely and efficiently. Adding login and registration features can enhance the user experience. User profiles should be managed, allowing users to update their information and customize their preferences. Personalization is essential for providing personalized suggestions based on interaction history and previous preferences. However, data collection must be transparent, in compliance with privacy laws, and users have the option to delete their data. The Python language and SQLite3 database can be used to implement user management in chatbots. Personalization features can be implemented using preference information stored in the database.

In [1]:
%pip install PyQt5

In [2]:
# Importing all the necessary libraries and resources:
import sys
import sqlite3
import hashlib
from PyQt5.QtWidgets import QApplication, QMainWindow, QLabel, QLineEdit, QPushButton, QVBoxLayout, QWidget, QMessageBox

In [3]:
# Connnecting to SQLite database:
conn = sqlite3.connect('usuarios.db')
cursor = conn.cursor()

# Creating 'users' table:
cursor.execute('''
CREATE TABLE IF NOT EXISTS users (
  id INTEGER PRIMARY KEY AUTOINCREMENT,
  nome TEXT NOT NULL,
  email TEXT NOT NULL UNIQUE,
  password TEXT NOT NULL,
  preference TEXT
)
''')

conn.commit()

In [1]:
# Function to encrypt the password (using SHA256):
def encrypt_password(password):
  return hashlib.sha256(password.encode()).hexdigest()

# Function to add new user:
def add_user(name, email, password, preference):
  encrypted_password = encrypt_password(password)
  try:
    cursor.execute('''
    INSERT INTO users (name, email, password, preference)
    VALUES (?, ?, ?, ?)
    ''', (name, email, encrypted_password, preference))
    conn.commit()
    return True
  except sqlite3.IntegrityError:
    return False

# Function for user log in:
def user_login(email, password):
  encrypted_password = encrypt_password(password)
  cursor.execute('''
  SELECT * FROM users WHERE email = ? AND password = ?
  ''', (email, encrypted_password))
  user = cursor.fetchone()
  return user

# Function to edit user profile:
def edit_profile(user_id, new_name=None, new_preference=None):
  if new_name:
    cursor.execute('UPDATE users SET name = ? WHERE id = ?', (new_name, user_id))
  if new_preference:
    cursor.execute('UPDATE users SET preference = ? WHERE id = ?', (new_preference, user_id))
    conn.commit()

# Function to personalize user experience:
def personalize_experience(user):
  preference = user[4]  # Position 4 where the preference is stored.
  if preference:
    return f'Hello {user[1]}, you like {preference}. Here goes some personalized options!'
  else:
    return f'Hello {user[1]}, you still have not configured your preferences.'

# Class for the main PyQt5 window:
class App(QMainWindow):
  def __init__(self):
    super().__init__()
    self.logged_user = None
    self.initUI()

NameError: name 'QMainWindow' is not defined

In [ ]:
def initUI(self):
    self.setWindowTitle('User System with PyQt5')
    self.setGeometry(100, 100, 400, 300)

    layout = QVBoxLayout()

    self.label = QLabel('Welcome to our User System,', self)
    layout.addWidget(self.label)

    self.input_email = QLineEdit(self)
    self.input_email.setPlaceholderText('Email')
    layout.addWidget(self.input_email)

    self.input_password = QLineEdit(self)
    self.input_password.setPlaceholderText('password')
    self.input_password.setEchoMode(QLineEdit.Password)
    layout.addWidget(self.input_password)

    self.login_btn = QPushButton('Login', self)
    self.login_btn.clicked.connect(self.log_in)
    layout.addWidget(self.login_btn)

    self.register_btn = QPushButton('register', self)
    self.register_btn.clicked.connect(self.register)
    layout.addWidget(self.register_btn)

    container = QWidget()
    container.setLayout(layout)
    self.setCentralWidget(container)

    def log_in(self):
      email = self.input_email.text()
      password = self.input_password.text()

      user = user_login(email, password)
      if user:
        self.logged_user = user
        self.label.setText(f'Welcome, {user[1]}!')
        self.open_profile_window()
      else:
        QMessageBox.warning(self, 'Erro', 'Wrong email or password.')

    def register(self):
      name = self.input_email.text()
      email = self.input_email.text()
      password = self.input_password.text()
      preference = 'books'  # Exemple of initial preference

      if add_user(name, email, password, preference):
        QMessageBox.information(self, 'Success', 'User was successfully registered!')
      else:
        QMessageBox.warning(self, 'Error', 'Error registering user. This email has already been registered.')

    def open_profile_window(self):
      self.input_email.hide()
      self.input_password.hide()
      self.login_btn.hide()
      self.register_btn.hide()

      self.label.setText(f'Hello {self.logged_user[1]}, you are logged in!')

      self.btn_edit_profile = QPushButton('Edit Profile', self)
      self.btn_edit_profile.clicked.connect(self.edit_profile)
      self.centralWidget().layout().addWidget(self.btn_edit_profile)

      self.btn_personalizar = QPushButton('Personalize Experience', self)
      self.btn_personalizar.clicked.connect(self.personalize_experience)
      self.centralWidget().layout().addWidget(self.btn_personalizar)

      def edit_profile(self):
        new_name, ok = QInputDialog.getText(self, 'Edit name', 'Enter your new name:')
        new_preference, ok = QInputDialog.getText(self, 'Edit Preference', 'Enter your new preference:')

        edit_profile(self.logged_user[0], new_name=new_name, new_preference=new_preference)
        QMessageBox.information(self, 'Success', 'Profile updated successfully!')

      def personalize_experience(self):
        message = personalize_experience(self.logged_user)
        QMessageBox.information(self, 'Personalization', message)

# Executing the app:
if __name__ == '__main__':
  app = QApplication(sys.argv)
  ex = App()
  ex.show()
  sys.exit(app.exec_())